**Sommario**
- [Web scraping: creare uno spider con Python](#web-scraping-creare-uno-spider-con-python)
  - [Spider](#spider)
    - [Crawler](#crawler)
    - [Scraper](#scraper)
  - [Ecosistema software per il web scraping (*maggio 2023*)](#ecosistema-software-per-il-web-scraping-*maggio-2023*)
    - [Scenario 1: Pagine statiche](#scenario-1-pagine-statiche)
      - [HTTP client](#http-client)
      - [HTML scraper](#html-scraper)
      - [HTTP client + HTML scraper](#http-client-+-html-scraper)
    - [Scenario 2: Pagine dinamiche](#scenario-2-pagine-dinamiche)
      - [Headless browser](#headless-browser)
      - [Ispezione delle pagine e del traffico di rete](#ispezione-delle-pagine-e-del-traffico-di-rete)
  - [Broswer tools](#broswer-tools)
  - [Contromisure per evitare di essere rilevati come "bot"](#contromisure-per-evitare-di-essere-rilevati-come-bot)
    - [Fare pause di durata casuale tra una richieste HTTP e la successiva](#fare-pause-di-durata-casuale-tra-una-richieste-http-e-la-successiva)
    - [Modificare l'User-Agent del client HTTP](#modificare-l'user-agent-del-client-http)
  - [Siti utili per fare pratica (sandbox)](#siti-utili-per-fare-pratica-sandbox)
  - [Request + Beautiful Soup](#request-+-beautiful-soup)
    - [Info e installazione di `beautifulsoup4`](#info-e-installazione-di-beautifulsoup4)
    - [Sito target](#sito-target)
    - [Request](#request)
    - [Pasrsing](#pasrsing)
    - [Ottenere un elemento specifico: oggetto `Tag`](#ottenere-un-elemento-specifico-oggetto-tag)
      - [Metodo `find()`](#metodo-find)
      - [Metodo `find_all()`](#metodo-find_all)
      - [Attributo-tag](#attributo-tag)
      - [Metodo `select()`](#metodo-select)
    - [Estrarre il testo di un tag](#estrarre-il-testo-di-un-tag)
    - [Estrarre il valore di un attributo](#estrarre-il-valore-di-un-attributo)
    - [Riassumendo](#riassumendo)
  - [Esempio di spider minimale](#esempio-di-spider-minimale)

# Web scraping: creare uno spider con Python

Fare *web scraping* significa estrarre dei dati dalle pagine web in modo mirato e registrarli in strutture dati a noi utili al fine di poterli manipolare e interrogare in modi più efficaci ed efficienti rispetto a come la nostra fonte ce li presenta sul web.

I software che fanno questi tipi di operazioni vengono detti *web spider*.

## Spider

Come un ragno si nuove sulla propria tela (*web*) per andare a cibarsi delle prede catturate, così uno *spider* è un software che si muove autonomamente sul web per andarsi a prendere i documenti da cui estrarre le informazioni.

Per fare tutto questo automaticamente, uno spider è solitamente composto da due parti, aventi funzioni specifiche: il _**crawler**_, che trova le risorse e lo _**scraper**_ che estrae le informazioni da queste risorse.

Se possediamo già un elenco di pagine, potremmo non necessitare del *crawler* ed estrarre direttamente i dati da ciascuna pagina.

### Crawler

Lo scopo del processo di *crowling*, è quello di scoprire nuove pagine e risorse. Per esempio il cuore di Google Search è un crawler che naviga sul web e indicizza tutto quello che trova, secondo determinate regole. Solitamente un crawler legge i link presenti in una pagina e decide se e come proseguire l'apertura di altre pagine.

Per esempio, se vogliamo estrarre tutti gli articoli in vendita corrispondenti a certe parole chiave da un sito di annunci, dobbiamo prima scrivere del codice che effettui la ricerca con quelle parole, e infine navighi tra le pagine dei risultati per recuperare i link di tutte le pagine trovate.

![crawling.gif](./imgs/http/crawling.gif)

### Scraper

Lo scopo del processo di *scraping*, è quello di estrarre le informazioni che ci servono, solitamente da un elenco di pagine trovate dal crawler.

Nell'esempio precedente, lo scraper esaminerà dunque, una a una, le pagine di ciascun articolo in vendita in modo da estrarre i dati che ci servono.

![scraping.gif](./imgs/http/scraping.webp)

## Ecosistema software per il web scraping (*maggio 2023*)

### Scenario 1: Pagine statiche
Se avete delle pagine i cui dati sono presenti nell'HTML, potete usare direttamente un client HTTP e un parser HTML. Per esempio l'accoppiata più comune è `requests` più `beautifulsoup4`.

In alternativa, `Scrapy` offre già una soluzione all-in-one, gestendo lui direttamente anche le chiamate HTTP.

Ecco i tool più utilizzati al momento, per manipolare pagine HTML statiche, divisi per categoria:

#### HTTP client

Per effettuare delle chiamate alle API dei siti target e ottenere le risorse che ci servono, come risposta:
- [`requests`](https://pypi.org/project/requests/) il modulo che già conosciamo. Sviluppato dalla Python Software Foundation.
- [`httpx`](https://pypi.org/project/httpx/) libreria di terze parti.
- [`aiohttp`](https://pypi.org/project/aiohttp/) libreria di terze parti.

#### HTML scraper

Per manipolare i documenti HTML come oggetti Python:
- [`beautifulsoup4`](https://pypi.org/project/beautifulsoup4/): la libreria storicamente più famosa ed utilizzata per manipolare HTML. Sviluppata e manutenuta da un certo Leonard Richardson, sviluppatore apparentemente molto *old school*.

Se volte un controllo più "a basso livello" potete usare direttmente un *parsing engine* come:
- [`parsel`](https://pypi.org/project/parsel/): motore di parsing HTML molto potente e prestante; è il motore usato da *Scrapy*. Sviluppato da Scrapinghub/Zyte.
- [`lxml`](https://pypi.org/project/lxml/): il modulo che già conosciamo, oltre all'XML può leggere anche HTML. Sviluppato inizialmente da un gruppo di programmatori tedeschi.

#### HTTP client + HTML scraper

Alcuni tool offrono già due strumenti in uno: HTTP client e manipolazione HTML. Scrapy è il più famoso, attualmente:

- [`Scrapy`](https://pypi.org/project/Scrapy/): la libreria più usata a livello professionale, studiata proprio per fare web scraping. Con essa si possono fare direttamente le chimate HTTP e il parsing dell'HTML. Usa `parsel` come motore di parsing. Sviluppato da Scrapinghub/Zyte.

### Scenario 2: Pagine dinamiche

I siti oggigiorno fanno tutti uso di JavaScript per rendere interattivi i contenuti delle pagine HTML. Inoltre JavaScript può essere usato per moltissimi scopi, e può alterare completamente il contenuto HTML di una pagina. Nei casi più estremi, potremmo ritrovarci con una pagina HTML completamente vuota, la cui struttura e contenuto vengono costruiti "al volo" tramite codice JavaScript.

Tuttavia è più comune ritrovarsi in una situazione intermedia, in cui parte della pagina è già presente nel file HTML che riceviamo, ma i dati (per esempio di una ricerca) vengono caricati in modo dinamico tramite JavaScript. In questo caso non ci basta scaricare la pagine HTML ma dobbiamo riuscire ad ottenere i dati che vengono caricati dinamicamente. In questo caso potete procedere in due modi:

- Usare un *headless browser* che esegua il codice JavaScript e renderizzi la pagina HTML finale così da poter fare lo scraping dei dati.

- Scoprire il modo in cui JavaScript ottiene i dati da inserire nella pagina e riprodurre le richieste verso il server nello stesso modo, così da ricevere direttamente i dati che ci interessano.

Parliamo innanzitutto del primo modo.

#### Headless browser

Per renderizzare l'HTML finale eseguendo il codice JavaScript che gestisce la pagina, bisogna ricorrere a un broswer. Non è necessario tuttavia aprire il browser vero e proprio, ma lo si esegue solitamente in modalità *headless*. Un [*headless broswer*](https://en.wikipedia.org/wiki/Headless_browser) è un browser eseguito senza un'interfaccia utente grafica, ma solo in modalità a riga di comando. Possiamo quindi usare Python per comunicare con il browser tramite delle API.

Esistono molte soluzioni diverse, ma tutte si appoggiano in ultima analisi a un motore JavaScript di un *headless broswer*:

- [`requests_html`](https://pypi.org/project/requests-html/): è un semplice pacchetto Python che può eseguire direttamente il codice JavaScript nelle pagine utilizzando `pyppeteer` (una libreria per usare Puppeteer sotto Python - vedi più avanti) e una versione di Chromium headless installata automaticamente. Si propone di offrire una soluzione *out-of-the-box* per le pagine dinamiche. Questa libreria è ancora un po' sperimentale e non viene aggiornata di frequente; è comunque da provare e tenere d'occhio dato che è sviluppata, come `requests`, dalla Python Software Foundation. 

- [Splash](https://splash.readthedocs.io/en/stable/index.html): scritto in Python, è un servizio di rendering JavaScript con API HTTP e un browser minimale. È pensato per essere eseguito in un container Docker ed è possibile personalizzarne il comportamento tramite degli script scritti in [Lua](https://it.wikipedia.org/wiki/Lua). È molto potente e lo possiamo considerare il coltellino svizzero e allo stesso tempo il bisturi per le operazioni di web scraping. Viene utilizzato a livello professionale ed è sviluppato da ScrapingHub/Zyte e finanziato in parte dal DARPA americano. Se usi `Scrapy`, c'è il modulo/plugin [`scrapy-splash`](https://pypi.org/project/scrapy-splash/) che facilita la comunicazione tra Python e il server docker di Splash.

- [`playwright`](https://pypi.org/project/playwright/): pacchetto Python basato sull'[omonima libreria](https://playwright.dev/) per node.js, permette di automatizzare Chromium, Firefox e Safari tramite il protocollo DevTools, compatibile con tutti i moderni browser. Sviluppato da Microsoft. Se usi `Scrapy`, c'è il modulo/plugin `scrapy-playwright` che facilita l'uso delle API di `playwright`.

- [`selenium`](https://pypi.org/project/selenium/) + WebDriver per il browser (es. geckodriver, chromedriver ecc.): permette di automatizzare i browser tramite il protocollo [WebDriver](https://en.wikipedia.org/wiki/Selenium_(software)#Selenium_WebDriver).

- [`pyppeteer`](https://pypi.org/project/pyppeteer/): pacchetto Python che usa la libreria per node.js [Puppeteer](https://pptr.dev/) per controllare il browser Chrome/Chromium tramite il protocollo DevTools. Sviluppato da Google. Consente grosso modo di fare le stesse cose che possiamo fare con `playwright`, ma solo su Chrome/Chromium.

Vi sono poi delle librerie solo per node.js, per le quali non esiste ancora alcun wrapper per Python:

- [Crawlee](https://crawlee.dev/): libreria per node.js che offre un pacchetto completo per fare web scraping su pagine dinamiche. Sviluppato da Apify. Attualmente non risultano progetti stabili che consentano l'uso di questa libreria JavaScript con Python. Può essere una buona scusa per installare node.js e buttarsi su JavaScript!

> ALLO STATO DELL'ARTE: Per lo scraping su pagine dinamiche, a maggio 2023, abbiamo Splash + Scrapy per uso professionale oppure Playwright + Scrapy per uso "casalingo". Selenium e Puppeteer li lasciamo come ultima chance.

#### Ispezione delle pagine e del traffico di rete

Un'alternativa è analizzare e scoprire il modo in cui JavaScript ottiene i dati da inserire nella pagina ed riprodurre le richieste verso il server nello stesso modo, così da ricevere direttamente i dati che ci interessano.

In questo caso dobbiamo usare strumenti di analisi del traffico di rete generato dal nostro browser e ricostruire i comportamenti della pagina che ci interessano. Gli strumenti per sviluppatori (DevTools) del nostro browser di solito sono sufficienti a questo scopo.

## Broswer tools

Per fare scraping efficacemente abbiamo bisogno anche di strumenti di controllo e analisi su uno o più browser, in modo da capire come funzionano le pagine da cui vogliamo estrarre informazioni. Per prima cosa, tutti i browser moderni dispongono di un'utility per questo scopo:

- [DevTools](https://en.wikipedia.org/wiki/Web_development_tools): *Developer tools*, ovvero gli "Strumenti per sviluppatori". Sono già presenti su tutti i principali browser (es. su [Chrome](https://developer.chrome.com/docs/devtools/overview/) o [Firefox](https://firefox-source-docs.mozilla.org/devtools-user/)). Sono molto utili perché ci permettono di ispezionare la pagina HTML, osservare le richieste HTTP generate dalla pagina e interagire con il codice JavaScript tramite una console interattiva (come l'interprete interattivo di Python).

Inoltre è importante poter inibire l'esecuzione degli script JavaScript a propria discrezione. I DevTools consentono di disattivare completamente JavaScript, ma se necessitiamo di inibire solo alcune fonti, lasciando eseguire gli script provenienti da altre, potrebbero tornarci utile uno di questi due plugin (non tutti e due perché vanno in conflitto):

- [NoScript](https://noscript.net/) - Consente di bloccare il caricamento dei file JavaScript in modo mirato. Per Firefox e Chrome.
- [uBlock Origin](https://ublockorigin.com/it) - Consente di bloccare il caricamento dei file JavaScript in modo mirato. Per Firefox e Chrome.
- [LibreJS](https://www.gnu.org/software/librejs/) - Consente di bloccare il caricamento dei file JavaScript in modo mirato. Per Firefox e Chrome. Un po' ostico da usare, ma molto preciso.

Vi consiglio di dare anche un'occhiata alle seguenti estensioni/plugin per i browser:

- [Wappalyzer](https://www.wappalyzer.com/apps/) - Rileva e profila le tecnologie usate su un sito web. Può servire per capire meglio come funziona una pagona. Per Firefox e Chrome.

## Contromisure per evitare di essere rilevati come "bot"

Non tutti i siti tollerano che vengano estratti dati dalle proprie pagine in modo automatizzato e sistematico tramite ciò che in gergo sono definiti "bot", contrazione di "robot".

I nostri spider sono un tipo di bot.

Innanzitutto devi essere consapevole che se hai intenzione di usare i dati estratti da un sito per scopi commerciali, dovresti leggere MOLTO BENE le condizioni di utilizzo dei servizi offerti da quel sito, e dovresti anche rispettare il contenuto di eventuali [file `robots.txt`](https://it.wikipedia.org/wiki/Protocollo_di_esclusione_robot). Per qualsiasi dubbio, è consigliabile la consulenza di un legale specializzato in "diritto digitale".

Detto questo, lo ripeto: Non tutti i siti tollerano che qualcuno faccia scraping tra le loro pagine web.

Nelle condizioni di utilizzo di un sito potrebbe essere indicato che l'estrazione dati, ovvero il web scraping, non è consentita. In questi casi di solito sono indicate anche le conseguenze per chi viola questa condizione. Solitamente le conseguenze possono essere:
- Blocco dell'IP da cui provengono le richieste HTTP.
- Blocco dell'eventuale account necessario per accedere al sito (es. social network, forum, e-commerce privati ecc.).

Dunque, se non volete rischiare di ritrovarvi con l'IP o l'account bannati, ci sono alcune accortezze che sarebbe meglio seguire:

- Non eseguire troppe richieste al secondo. Meglio eseguire richieste con un intervallo casuale (ad es. tra 2 e 5 secondi).
- Modificare l'header HTTP "User-Agent" e "camuffarsi" da comune browser.

Queste due semplici accortezze sono efficaci nella maggior parte dei casi; tuttavia i siti più famosi e visitati adottano spesso [tecniche di rilevazione dei bot](https://en.wikipedia.org/wiki/Bot_prevention) più sofisticate.

In questi casi, potrebbe essere utile una o più di queste "contromisure per le contromisure":

- Utilizzare la classe `Session` di `requests` in modo da memorizzare i cookie che invia il sito, comportandosi così come un bravo browser.
- Simulare in modo più preciso gli header HTTP che inviano i normali browser. Oltre all'User-Agent ci sono molti altri header che forniscono al il server informazioni le quali, se mancanti, potrebbero far presupporre l'uso di uno spider.

Se anche questo non è sufficiente, allora potrebbe essere il caso di utilizzare un browser headless con `Splash`, `playwright` o `selenium` e provare questi altri metodi:

- Eseguire il codice JavaScript.
- Eseguire il render completo della la pagina.
- Riprodurre movimenti del mouse casuali sulla pagina.
- Riprodurre movimenti del mouse mirati a compiere determinate azioni sulla pagina.

In alternativa, se il vostro IP continua ad venire bannato, potresti aver bisogno di un *rotating proxy*, ovvero un [proxy](https://it.wikipedia.org/wiki/Proxy) che vi assegna un nuovo IP ad ogni nuova sessione di navigazione.

Se inveve il sito risulta inaccessibile dalla vostra localizzazione geografica, potreste dover usare una [VPN](https://it.wikipedia.org/wiki/Rete_privata_virtuale) che vi faccia navigare come se vi trovaste in un altra nazione.

Vediamo ora come implementare i primi due semplici accorgimenti:

### Fare pause di durata casuale tra una richieste HTTP e la successiva

Per ottenere dei numeri casuali:

In [ ]:
from random import randint, uniform, random

print(random())
print(random() * 10)
print(randint(1, 10))
print(uniform(1, 10))


0.6513967996308621
9.528442793256259
5
3.633075494916878


Per fare una pausa, che sospende l'esecuzione del codice in modo temporaneo, possiamo usare la funzione `sleep()` dal modulo built-in `time`:

In [ ]:
from time import sleep
from random import uniform

for n in range(10):
    random_seconds = uniform(1, 4)
    print(f'Pippo, aspetta {random_seconds} secondi!')
    sleep(random_seconds)

Pippo, aspetta 1.286189290660835 secondi!
Pippo, aspetta 1.4352342639453244 secondi!
Pippo, aspetta 3.5572415906105332 secondi!
Pippo, aspetta 1.4090973918104175 secondi!
Pippo, aspetta 2.0534323160500527 secondi!
Pippo, aspetta 1.2951963579882193 secondi!
Pippo, aspetta 2.0020739149494458 secondi!
Pippo, aspetta 3.703189642465103 secondi!
Pippo, aspetta 3.220969400488466 secondi!
Pippo, aspetta 3.623118475110846 secondi!


### Modificare l'User-Agent del client HTTP

Se volete usare gli User-Agent più comuni e più recenti, li potete trovare qua:

- User-Agent di [Firefox](https://www.whatismybrowser.com/guides/the-latest-user-agent/firefox)
- User-Agent di [Chrome](https://www.whatismybrowser.com/guides/the-latest-user-agent/chrome)

Esistono anche molti siti che offrono elenchi degli User-Agent noti, basta fare qualche [ricerca sul web](https://www.google.com/search?q=user+agent+list).

Tuttavia come abbiamo già detto, un normale browser invia molti altri dati negli header della richiesta, come per esempio informazioni sui cookie e altre utili al funzionamento del sito. Quindi, se il server a cui vi state collegando necessita di particolari header, dovete provvedere voi a impostarli in modo dorretto prima di effettuare la richiesta.

In [8]:
from pprint import pprint
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:109.0) Gecko/20100101 Firefox/113.0'
}

response = requests.get('https://scrapeme.live/shop/', headers=headers)

pprint(dict(response.request.headers), width=100)

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate, br',
 'Connection': 'keep-alive',
 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:109.0) Gecko/20100101 Firefox/113.0'}


## Siti utili per fare pratica (sandbox)

Per fare pratica e provare a eseguire richieste in un ambiente di prova, ecco alcuni siti utili:

- https://toscrape.com/
- https://quotes.toscrape.com/scroll
- https://www.scrapethissite.com/
- https://scrapeme.live/shop/
- https://httpbin.org/

## Request + Beautiful Soup

Se avete delle pagine i cui dati sono presenti nell'HTML ricevuto con `requests`, potete usare direttamente `beautifulsoup4`.

### Info e installazione di `beautifulsoup4`

- [Sito ufficiale](https://www.crummy.com/software/BeautifulSoup/)
- [Documentazione](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).
- [Pagina su PyPi](https://pypi.org/project/beautifulsoup4/) (Python Package Index).
- [Repo su Launchpad](https://git.launchpad.net/beautifulsoup/tree/).

Beautiful Soup è una libreria Python per estrarre dati da file HTML e XML in modo semplice e (abbastanza) intuitivo.

Questa è una libreria di terze parti e dunque non fa parte della Libreria standard. È necessario installarla tramite il comando `pip install beautifulsoup4` nella riga di comando.

```bash
# MAC/LINUX:
(my_venv) $ pip install beautifulsoup4

# WINDOWS:
(my_venv) C:\my_proj> pip install beautifulsoup4
```


Se sei su Windows, non hai creato un virtual environment (male!;) e devi usare il `py` launcher:

```powershell
C:\my_proj> py -m pip install beautifulsoup4
```

Per iniziare a usare requests nel vostro codice, importate la libreria:

```python
import bs4
```

Oppure, dato che di solito ci serve solo la classe `BeautifulSoup`, meglio:

```python
from bs4 import BeautifulSoup
```

### Sito target

Per allenarsi a fare scraping possiamo iniziare da uno dei [siti "sandbox"](#siti-utili-per-fare-pratica-sandbox) che abbiamo elencato poco prima.

In questo tutorial useremo il sito https://scrapeme.live/shop/ che riproduce un finto negozio online di Pokemon.

Vediamo come poter estrarre l'elenco degli articoli in vendita e i relativi dati.

### Request

Per prima cosa, è necessario creare una variabile che memorizzi il contenuto della pagina. Per farlo, utilizziamo il modulo `requests` e facciamo una chiamata GET all'URL della pagina che ci interessa.

CONSIGLIO: Inizialmente, per scrivere la prima parte del voststro scraper, parti sempre da una pagina singola, anche se ne hai molte da analizzare. Quando sarete riusciti ad estrarre i contenuti di quella pagina, allora procedete ad analizzare le successive.

Per verificare se la pagina è stata scaricata con successo, si può controllare l'attributo `status_code` della risposta. Se il codice restituito è `200`, non ci sono errori. Se il codice è `4xx` o `5xx`, ci sono delle difficoltà nell'ottenere la pagina.

### Pasrsing

Una volta che abbiamo ottenuto la pagina web, utilizziamo la classe `BeautifulSoup` per fare il [parsing](https://it.wikipedia.org/wiki/Parsing) dell'HTML contenuto nella pagine e ottenere un oggetto Python detto *parse tree* o "[albero sintattico](https://it.wikipedia.org/wiki/Albero_sintattico)", una struttura dati avente appunto una struttura ad albero. Quest'oggetto rappresenta la pagina, il *document*, nel suo complesso e ci consente di interagire con ogni sua parte.

In [43]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://scrapeme.live/shop/')

if response.status_code == 200:

    soup = BeautifulSoup(response.content, 'html.parser')

    print(type(soup))
    print(soup.name)

<class 'bs4.BeautifulSoup'>
[document]


Qui sopra, abbiamo passato alla classe `BeautifulSoup` due parametri: `response.content` è il contenuto raw della pagina e `'html.parser'` è il parser di default incluso nella Libreria standard di Python. È possibile installare parser aggiuntivi come `lxml` tramite `pip` e utilizzarli al posto di `'html.parser'`. Si dice che il parser di `lxml` sia un po' più veloce ma meno flessibile.

In questo esempio, la nostra variabile `soup` ora contiene un oggetto di tipo `bs4.BeautifulSoup` che rappresenta l'intero documento ed ha una struttura ad albero. È possibile utilizzare il metodo `.prettify()` per visualizzare i dati con le giuste indentazioni.

In [37]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-GB">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=2.0" name="viewport"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://scrapeme.live/xmlrpc.php" rel="pingback"/>
  <title>
   Products – ScrapeMe
  </title>
  <link href="//fonts.googleapis.com" rel="dns-prefetch">
   <link href="//s.w.org" rel="dns-prefetch">
    <link href="https://scrapeme.live/feed/" rel="alternate" title="ScrapeMe » Feed" type="application/rss+xml"/>
    <link href="https://scrapeme.live/comments/feed/" rel="alternate" title="ScrapeMe » Comments Feed" type="application/rss+xml"/>
    <link href="https://scrapeme.live/shop/feed/" rel="alternate" title="ScrapeMe » Products Feed" type="application/rss+xml"/>
    <script type="text/javascript">
     window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/11\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/11\

### Ottenere un elemento specifico: oggetto `Tag`

Come si può vedere, il contenuto della variabile `soup` è l'intero documento HTML, il quale è difficile da interpretare così a prima vista; inoltre contiene molte informazioni che non ci interessano e quelle che ci interessano sono sparse qua e là.

I dati importanti, come i testi o i titoli, sono spesso memorizzati con tag particolari. Quindi, una volta individuati quali tag contengono i dati che vogliamo estrarre, possiamo procedere a crercarli, interrogando il nostro *parse tree*. Ci sono più metodi che ci consentono di trovare gli elementi HTML.

Ciascun **tag** rappresenta un singolo **elemento** HTML. Nel gergo del [DOM](https://it.wikipedia.org/wiki/Document_Object_Model e JavaScipt è detto "_**node**_". In BeautifulSoup è rappresentato da un oggetto di tipo `Tag` e ciascun tag HTML, proprio come quello XML, può contenere attributi e altri tag.

L'oggetto `BeautifulSoup` lo possiamo considerare come il tag "top-level" (che tra l'altro è `<html>`), per cui possiede gli stessi metodi di interrogazione/ricerca di un qualunque altro oggetto `Tag`.

Abbiamo detto che la pagina che abbiamo scaricato riproduce un finto store online di Pokemon. Analizzando l'HTML della pagina notiamo che ciascun prodotto, ciascun Pokemon, è contenuto in un tag `<li>` di una elenco HTML `<ul>`.

Ecco un singolo Pokemon:

```html
<li class="post-759 product type-product status-publish has-post-thumbnail product_cat-pokemon product_cat-seed product_tag-bulbasaur product_tag-overgrow product_tag-seed first instock sold-individually taxable shipping-taxable purchasable product-type-simple">
   <a class="woocommerce-LoopProduct-link woocommerce-loop-product__link" href="https://scrapeme.live/shop/Bulbasaur/">
      <img alt="" class="attachment-woocommerce_thumbnail size-woocommerce_thumbnail wp-post-image" height="324" sizes="(max-width: 324px) 100vw, 324px" src="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png" srcset="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png 350w, https://scrapeme.live/wp-content/uploads/2018/08/001-150x150.png 150w, https://scrapeme.live/wp-content/uploads/2018/08/001-300x300.png 300w, https://scrapeme.live/wp-content/uploads/2018/08/001-100x100.png 100w, https://scrapeme.live/wp-content/uploads/2018/08/001-250x250.png 250w, https://scrapeme.live/wp-content/uploads/2018/08/001.png 475w" width="324"/>
      <h2 class="woocommerce-loop-product__title">Bulbasaur</h2>
      <span class="price"><span class="woocommerce-Price-amount amount"><span class="woocommerce-Price-currencySymbol">£</span>63.00</span></span>
   </a>
   <a aria-label="Add “Bulbasaur” to your basket" class="button product_type_simple add_to_cart_button ajax_add_to_cart" data-product_id="759" data-product_sku="4391" data-quantity="1" href="/shop/?add-to-cart=759" rel="nofollow">Add to basket</a>
</li>
```

Vediamo ora come trovare questi tag `<li>`

#### Metodo `find()`

Il metodo `find()` restituisce la prima occorrenza del tag nell'albero. Questo metodo è adatto se si è sicuri che il documento abbia un solo tag specifico.

In caso non venga trovato nessun elemento, il metodo `find()` restituisce `None`

In [39]:
li_elem = soup.find('li')  

# Controlliamo il contenuto dell'elemento
print(li_elem)

# Controlliamo il tipo dell'elemento
print(type(li_elem))


<li><a href="https://scrapeme.live/">Home</a></li>
<class 'bs4.element.Tag'>


Questo metodo è poco utile nel nostro caso, in quanto vogliamo ottenere più elementi. Inoltre, indicando solo un generico tag `<li>` è stato trovato un elemento che non ci interessa.

Questo metodo può essere utile se invocato su un tag particolare, per ottenere un elemento che sappiamo essere unico al suo interno. Se invocato sul documento `BeautifulSoup`, potremmo usarlo per accedere velocemente a tag come `<title>`, `<head>`, `<body>` ecc.

In [43]:
title_elem = soup.find('title')  

print(title_elem)

<title>Products – ScrapeMe</title>


#### Metodo `find_all()`

Il metodo `find_all()` restituisce una lista di tutti i risultati con il tag ricercato.

In caso non venga trovato nessun elemento, il metodo `find_all()` restituisce una lista vuota `[ ]`.

In [17]:
list_items = soup.find_all('li')
print(list_items)

[<li><a href="https://scrapeme.live/">Home</a></li>, <li><a href="https://scrapeme.live/">Home</a></li>, <li class="">
<a class="cart-contents" href="https://scrapeme.live" title="View your shopping basket">
<span class="woocommerce-Price-amount amount"><span class="woocommerce-Price-currencySymbol">£</span>0.00</span> <span class="count">0 items</span>
</a>
</li>, <li>
<div class="widget woocommerce widget_shopping_cart"><div class="widget_shopping_cart_content"></div></div> </li>, <li><span aria-current="page" class="page-numbers current">1</span></li>, <li><a class="page-numbers" href="https://scrapeme.live/shop/page/2/">2</a></li>, <li><a class="page-numbers" href="https://scrapeme.live/shop/page/3/">3</a></li>, <li><a class="page-numbers" href="https://scrapeme.live/shop/page/4/">4</a></li>, <li><span class="page-numbers dots">…</span></li>, <li><a class="page-numbers" href="https://scrapeme.live/shop/page/46/">46</a></li>, <li><a class="page-numbers" href="https://scrapeme.live/s

In questo modo però vengono individuati anche altri tag `<li>` che non ci interessano. Per ovviare a questo problema, possiamo anche specificare un filtro aggiuntivo passando degli attributi, come ad esempio `class`, `style` ecc., sotto forma di dizionario:

In [31]:
list_item_products = soup.find_all('li', {'class': 'product'})
print(list_item_products)

[<li class="post-759 product type-product status-publish has-post-thumbnail product_cat-pokemon product_cat-seed product_tag-bulbasaur product_tag-overgrow product_tag-seed first instock sold-individually taxable shipping-taxable purchasable product-type-simple">
<a class="woocommerce-LoopProduct-link woocommerce-loop-product__link" href="https://scrapeme.live/shop/Bulbasaur/"><img alt="" class="attachment-woocommerce_thumbnail size-woocommerce_thumbnail wp-post-image" height="324" sizes="(max-width: 324px) 100vw, 324px" src="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png" srcset="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png 350w, https://scrapeme.live/wp-content/uploads/2018/08/001-150x150.png 150w, https://scrapeme.live/wp-content/uploads/2018/08/001-300x300.png 300w, https://scrapeme.live/wp-content/uploads/2018/08/001-100x100.png 100w, https://scrapeme.live/wp-content/uploads/2018/08/001-250x250.png 250w, https://scrapeme.live/wp-content/upload

Nell'esempio qua sopra abbiamo quindi modificato un po' la ricerca aggiungendo come secondo parametro un dizionario che specifica tutti gli elementi con tag `li` aventi la parola `product` nell'attributo `class`. In pratica le chiavi di questo dizionario sono gli attributi dei tag.

Come potete vedere, l'attributo `class` contiene anche molti altri valori separati da uno spazio vuoto; in altre parole l'elemento HTML può anche appartenere ad altre classi, nonostante ciò l'elemento viene trovato perché appartiene ANCHE alla classe `product`.

Il metodo `.find_all()` consente anche l'utilizzo di espressioni regolari, può ricevere una lista di tag da cercare o una funzione da utilizzare come filtro. Per approfondire l'argomento, [leggi sulla guida ufficuale](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree).

#### Attributo-tag

Un altra modalità per ottenere un determinato tag è quello di scrivere il tag come attributo di un oggetto di tipo `Tag` (o anche `BeautifulSoup`).

Se ci sono più tag con lo stesso nome, il metodo però restituirà solo la prima occorrenza.

In [40]:
print(soup.li)

<li><a href="https://scrapeme.live/">Home</a></li>


Come per `.find()`, questo metodo non è molto utile, a meno che non si voglia ottenere un elemento che sappiamo essere unico nella pagina o all'interno di un elemento speficico.

In [51]:
print(soup.head.title)

Products – ScrapeMe


#### Metodo `select()`

Un altro modo per trovare i nostri tag è tramite i cosiddetti *CSS selectors* (selettori CSS) usando il metodo `.select()`.

NOTA: L'effettiva implementazione per i selettori CSS si appoggia al pacchetto Soup Sieve, disponibile su PyPI come `soupsieve`. Se hai installato Beautiful Soup tramite `pip`, esso è stato già installato come dipendenza, quindi non è necessario fare altro.

L'elenco dei selettori CSS standard lo potete trovare sul [sito del W3C](https://www.w3.org/TR/selectors-3/) e [qua avete un riassunto](https://www.w3schools.com/cssref/css_selectors.php).

In [2]:
list_item_products = soup.select('li.product')
print(list_item_products)

[<li class="post-759 product type-product status-publish has-post-thumbnail product_cat-pokemon product_cat-seed product_tag-bulbasaur product_tag-overgrow product_tag-seed first instock sold-individually taxable shipping-taxable purchasable product-type-simple">
<a class="woocommerce-LoopProduct-link woocommerce-loop-product__link" href="https://scrapeme.live/shop/Bulbasaur/"><img alt="" class="attachment-woocommerce_thumbnail size-woocommerce_thumbnail wp-post-image" height="324" sizes="(max-width: 324px) 100vw, 324px" src="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png" srcset="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png 350w, https://scrapeme.live/wp-content/uploads/2018/08/001-150x150.png 150w, https://scrapeme.live/wp-content/uploads/2018/08/001-300x300.png 300w, https://scrapeme.live/wp-content/uploads/2018/08/001-100x100.png 100w, https://scrapeme.live/wp-content/uploads/2018/08/001-250x250.png 250w, https://scrapeme.live/wp-content/upload

### Estrarre il testo di un tag

Ora che conosciamo alcune nozioni di base di HTML e BeautifulSoup, è il momento di estrarre tutti i dati necessari.

Ora, per elaborare tutti i tag di questo elenco di prodotti, si può usare un ciclo `for` per iterare ciascun elemento e il metodo `.get_text()` o l'attributo `.text` per ottenere i dati testuali al suo interno.

In [5]:
list_item_products = soup.find_all('li', {'class': 'product'})

for li in list_item_products:
    print(soup.text)
    # print(li.get_text())


Bulbasaur
£63.00
Add to basket

Ivysaur
£87.00
Add to basket

Venusaur
£105.00
Add to basket

Charmander
£48.00
Add to basket

Charmeleon
£165.00
Add to basket

Charizard
£156.00
Add to basket

Squirtle
£130.00
Add to basket

Wartortle
£123.00
Add to basket

Blastoise
£76.00
Add to basket

Caterpie
£73.00
Add to basket

Metapod
£148.00
Add to basket

Butterfree
£162.00
Add to basket

Weedle
£25.00
Add to basket

Kakuna
£148.00
Add to basket

Beedrill
£168.00
Add to basket

Pidgey
£159.00
Add to basket


La differenza tra `li.text` e `li.get_text()` è che il metodo consente di passare degli argomenti aggiuntivi, mentre il semplice attributo, no.

### Estrarre il valore di un attributo

Per ottenere gli attributi di un tag possiamo fare come per i dizionari, usando la notazione a subscription `[key]` oppure il più sicuro metodo `.get(key)` (che, se ti ricordi, consente anche di impostare un valore di default in caso la chiave non esistesse).

Per esempio gli URL dei link sono contenuti nell'attributo `href` dei tag `<link>`.

Nel nostro esempio con il negozio online, recuperare i link può essere utile perché possiamo utilizzare gli URL dei vari prodotti in vendita per aprire le rispettive pagine e raccogliere altri dati che non sono presenti nelle pagine che elencano i Pokemon in vendita.

In [6]:
list_item_products = soup.find_all('li', {'class': 'product'})

for li in list_item_products:
    print(li.a.get('href')) 
    # print(li.a['href']) 

https://scrapeme.live/shop/Bulbasaur/
https://scrapeme.live/shop/Bulbasaur/
https://scrapeme.live/shop/Ivysaur/
https://scrapeme.live/shop/Ivysaur/
https://scrapeme.live/shop/Venusaur/
https://scrapeme.live/shop/Venusaur/
https://scrapeme.live/shop/Charmander/
https://scrapeme.live/shop/Charmander/
https://scrapeme.live/shop/Charmeleon/
https://scrapeme.live/shop/Charmeleon/
https://scrapeme.live/shop/Charizard/
https://scrapeme.live/shop/Charizard/
https://scrapeme.live/shop/Squirtle/
https://scrapeme.live/shop/Squirtle/
https://scrapeme.live/shop/Wartortle/
https://scrapeme.live/shop/Wartortle/
https://scrapeme.live/shop/Blastoise/
https://scrapeme.live/shop/Blastoise/
https://scrapeme.live/shop/Caterpie/
https://scrapeme.live/shop/Caterpie/
https://scrapeme.live/shop/Metapod/
https://scrapeme.live/shop/Metapod/
https://scrapeme.live/shop/Butterfree/
https://scrapeme.live/shop/Butterfree/
https://scrapeme.live/shop/Weedle/
https://scrapeme.live/shop/Weedle/
https://scrapeme.live/shop

### Riassumendo

Rassumiamo ora i vari metodi visti, provando a mettere tutte o assieme e stampare solo i dati che ci interessano.

In [50]:
list_item_products = soup.find_all('li', {'class': 'product'})

for li in list_item_products:
    
    print(li.h2.text)
    
    print(li.find('span', {'class': 'price'}).text)
    
    print(li.a.get('href')) 

    print('------------------')

Bulbasaur
£63.00
https://scrapeme.live/shop/Bulbasaur/
------------------
Ivysaur
£87.00
https://scrapeme.live/shop/Ivysaur/
------------------
Venusaur
£105.00
https://scrapeme.live/shop/Venusaur/
------------------
Charmander
£48.00
https://scrapeme.live/shop/Charmander/
------------------
Charmeleon
£165.00
https://scrapeme.live/shop/Charmeleon/
------------------
Charizard
£156.00
https://scrapeme.live/shop/Charizard/
------------------
Squirtle
£130.00
https://scrapeme.live/shop/Squirtle/
------------------
Wartortle
£123.00
https://scrapeme.live/shop/Wartortle/
------------------
Blastoise
£76.00
https://scrapeme.live/shop/Blastoise/
------------------
Caterpie
£73.00
https://scrapeme.live/shop/Caterpie/
------------------
Metapod
£148.00
https://scrapeme.live/shop/Metapod/
------------------
Butterfree
£162.00
https://scrapeme.live/shop/Butterfree/
------------------
Weedle
£25.00
https://scrapeme.live/shop/Weedle/
------------------
Kakuna
£148.00
https://scrapeme.live/shop/Kak

In questa sezione abbiamo imparato le principali operazioni con Beautiful Soup:

- creare un *parse tree*;
- cercare i tag in base al loro nome o al valore dei loro attributi;
- estrarre testo e attributi di un tag.

Beautiful Soup può apparire inizialmente molto complicato, ma dopo un po' di pratica ci si abituerà a questa libreria e presto si sarà in grado di raccogliere una grande varietà di dati. Per saperne di più su Beautiful Soup, consultate la sua [documentazione ufficiale](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

## Esempio di spider minimale

Proviamo a immaginare la struttura di uno scraper che estragga gli articoli dalle pagine che elencano i prodotti in vendita nel nostro finto negozio online di Pokemon.

In [6]:
from time import sleep
from random import randint
import requests
from bs4 import BeautifulSoup

# Parto da una lista di URL, inizialmente hard-coded. Poi posso provare a
# generare gli URL in modo automatico modificando il path o l'eventuale query string)
list_of_urls = [
    'https://scrapeme.live/shop/page/1/',
    'https://scrapeme.live/shop/page/2/',
    'https://scrapeme.live/shop/page/3/',
]

## Definisco la funzione di scraping per una singola pagina
def scrape_page(url):

    # Creo un contenitore per i dati dei singoli prodotti di questa singola pagina    
    items = []

    ## Invio la richiesta HTTP senza controllo del certificato SSL
    response = requests.get(url, verify=False)
    
    # Se la pagina esiste
    if response.status_code == 200:
        
        # Creo l'oggetto "parse tree"
        soup = BeautifulSoup(response.content, 'html.parser')
        # Cerco i dati che mi interessano e man mano li aggiungiamo al contenitore items
        ...

        ## Restituisce i dati trovati
        return items

# Creo un contenitore per raccogliere i dati delle varie pagine
scraped_data = []     

## Ciclo le pagine da analizzare
for url in list_of_urls:
    # Ottengo i dati di una singola pagina
    results = scrape_page(url)
    # Aggiungo i dati trovati al contenitore
    scraped_data += results
    # Aspetto un numero di secondi variabile tra 1 e 4
    sleep(randint(1, 4))

# Controllo i dati trovati
print(scraped_data)

/Users/knox/Repositories/python_training/python_corso_base/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scrapeme.live'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/knox/Repositories/python_training/python_corso_base/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scrapeme.live'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/knox/Repositories/python_training/python_corso_base/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scrapeme.live'. Adding certificate verification is strongly advised. See: ht

[]


Proviamo ora a scrivere del codice all'interno della funzione `scrape_page()`, in modo da estrarre "nome", "prezzo" e "url" di ogni Pokemon in vendita, mettendoli in un dizionario.

Ciascuno di questi dizionari creati viene poi aggiunto al contenitore `items` all'interno della funzione, il quale poi sarà restituito e aggiunto a sua volta agli altri dati trovati.

In [41]:
from pprint import pprint
from time import sleep
from random import uniform
import requests
from bs4 import BeautifulSoup

# Parto da una lista di URL, inizialmente hard-coded. Poi posso provare a
# generare gli URL in modo automatico modificando il path o l'eventuale query string)
list_of_urls = [
    'https://scrapeme.live/shop/page/1/',
    'https://scrapeme.live/shop/page/2/',
    'https://scrapeme.live/shop/page/3/',
]

## Definisco la funzione di scraping per una singola pagina
def scrape_page(url):

    # Creo un contenitore per i dati di questa singola pagina    
    items = []

    ## Invio la richiesta HTTP
    response = requests.get(url)
    
    # Se la pagina esiste
    if response.status_code == 200:
        
        # Creo l'oggetto "parse tree"
        soup = BeautifulSoup(response.content, 'html.parser')
        # Cerco i dati che mi interessano e man mano li aggiungiamo al contenitore items
        list_item_products = soup.find_all('li', {'class': 'product'})
        for li in list_item_products:
            data = {
                'url': li.a.get('href'),
                'name': li.h2.text,
                'price': li.find('span', {'class': 'price'}).text
            }
            items.append(data)

        ## Restituisce i dati trovati
        return items

# Creo un contenitore per raccogliere i dati delle varie pagine
scraped_data = []     

## Ciclo le pagine da analizzare
for url in list_of_urls:
    # Ottengo i dati di una singola pagina
    results = scrape_page(url)
    # Aggiungo i dati trovati al contenitore
    scraped_data += results
    # Aspetto un numero di secondi variabile tra 1 e 4
    sleep(uniform(1, 4))


# Controllo i dati trovati
pprint(scraped_data)

[{'name': 'Bulbasaur',
  'price': '£63.00',
  'url': 'https://scrapeme.live/shop/Bulbasaur/'},
 {'name': 'Ivysaur',
  'price': '£87.00',
  'url': 'https://scrapeme.live/shop/Ivysaur/'},
 {'name': 'Venusaur',
  'price': '£105.00',
  'url': 'https://scrapeme.live/shop/Venusaur/'},
 {'name': 'Charmander',
  'price': '£48.00',
  'url': 'https://scrapeme.live/shop/Charmander/'},
 {'name': 'Charmeleon',
  'price': '£165.00',
  'url': 'https://scrapeme.live/shop/Charmeleon/'},
 {'name': 'Charizard',
  'price': '£156.00',
  'url': 'https://scrapeme.live/shop/Charizard/'},
 {'name': 'Squirtle',
  'price': '£130.00',
  'url': 'https://scrapeme.live/shop/Squirtle/'},
 {'name': 'Wartortle',
  'price': '£123.00',
  'url': 'https://scrapeme.live/shop/Wartortle/'},
 {'name': 'Blastoise',
  'price': '£76.00',
  'url': 'https://scrapeme.live/shop/Blastoise/'},
 {'name': 'Caterpie',
  'price': '£73.00',
  'url': 'https://scrapeme.live/shop/Caterpie/'},
 {'name': 'Metapod',
  'price': '£148.00',
  'url':

Questo script potrebbe essere migoiorato generando in automatico le pagine successive alla 3.

Inoltre si potrebbe aprire ciascun prodotto usando il link ottenuto ed estrarre altri dati utili.